In [ ]:
%cd ..

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import glob
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import quantities as pq

import epych

In [ ]:
%matplotlib inline

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
CONDITIONS = ["go_gloexp", "go_seqctl", "lo_gloexp"]
PRETRIAL_SECONDS = 0.5
POSTTRIAL_SECONDS = 0.5

In [ ]:
def hippocampal_channels(channels):
    for l in channels.location.values:
        l = l.decode()
        yield ("DG-" in l or "CA" in l)

def visual_channels(channels):
    for l in channels.location.values:
        l = l.decode()
        yield ("VIS" in l)

def subcortical_channels(channels):
    for l in channels.location.values:
        l = l.decode()
        yield ("DG-" in l or "CA" in l or "MB" in l or "SCi" in l or "POST" in l)

In [ ]:
NWB_FILES = glob.glob('/mnt/data/*.nwb')

In [ ]:
PILOT_FILES = []

In [ ]:
evoked = {}

In [ ]:
for cond in CONDITIONS:
    for f, nwb_file in enumerate(NWB_FILES):
        basename = os.path.splitext(nwb_file)[0]
        if not os.path.exists(basename + "_" + cond):
            continue
        sampling = epych.recording.Sampling.unpickle(basename + "_" + cond).smap(lambda sig: sig.median_filter())
        evoked[(cond, nwb_file, True)] = sampling.baseline_correct(0., POSTTRIAL_SECONDS).erp()
        del sampling
        logging.info("Loaded LFPs for %s in subject %s" % (cond, nwb_file))

In [ ]:
for cond in CONDITIONS:
    logging.info("Plotting baseline-corrected LFPs for %s across subjects" % cond)
    for f, nwb_file in enumerate(NWB_FILES):
        basename = os.path.splitext(nwb_file)[0]
        if (cond, nwb_file, True) not in evoked:
            continue
        erp = evoked[(cond, nwb_file, True)]
        events = {
            "Oddball Onset": (erp.trials["stim3_start"].mean(), 'lightgreen'),
            "Oddball Offset": (erp.trials["stim3_end"].mean(), 'red'),
        }
        cortical = erp.smap(lambda sig: sig.select_channels(list(visual_channels(sig.channels))))
        cortical.plot(vmin=-1e-4, vmax=1e-4, figure="%s_%s_visual_lfp.pdf" % (basename, cond), layout="tight", **events)

        subcortical = erp.smap(lambda sig: sig.select_channels(list(subcortical_channels(sig.channels))))
        subcortical.plot(vmin=-1e-4, vmax=1e-4, figure="%s_%s_subcortical_lfp.pdf" % (basename, cond), layout="tight", **events)

In [ ]:
for cond in CONDITIONS:
    logging.info("Plotting baseline-corrected CSDs for %s across subjects" % cond)
    for f, nwb_file in enumerate(NWB_FILES):
        basename = os.path.splitext(nwb_file)[0]
        if (cond, nwb_file, True) not in evoked:
            continue
        erp = evoked[(cond, nwb_file, True)].smap(lambda sig: sig.downsample(4).current_source_density(depth_column="vertical"))
        events = {
            "Oddball Onset": (erp.trials["stim3_start"].mean(), 'lightgreen'),
            "Oddball Offset": (erp.trials["stim3_end"].mean(), 'red'),
        }
        cortical = erp.smap(lambda sig: sig.select_channels(list(visual_channels(sig.channels))))
        cortical.plot(vmin=-1e-4, vmax=1e-4, figure="%s_%s_visual_csd.pdf" % (basename, cond), **events)

        subcortical = erp.smap(lambda sig: sig.select_channels(list(subcortical_channels(sig.channels))))
        subcortical.plot(vmin=-1e-4, vmax=1e-4, figure="%s_%s_subcortical_csd.pdf" % (basename, cond), **events)